# CONFIGURATION ZONE (MANDATORY)

In [ ]:
#SETUP
Dir_X = "newDATA/X/"
Output_Dir_X = "newDATA/X_output/"

Dir_Y = "newDATA/Y/"
Output_Dir_Y = "newDATA/Y_output/"

Dir_Model = "src/X_output/"
Multithread_gran = 6

Export_name = "merged_forHDFS2"

HDFS_Dir = "/user/cluster/AdLearn/Debug/17-05-18/"


# Generate Model Reference

In [ ]:
import glob
import numpy as np
import pandas as pd

########## SCAN AND CONSTRUCT MATRIX FORM ######################
filesList = glob.glob(Dir_X + "*.csv")

ix = []
col = []
indexList = np.array(ix)
colList = np.array(col)

for f_ in filesList:
    print(f_)
    df = pd.read_csv(f_, index_col=0)
    indexList = np.append(indexList, df.index.values)
    indexList = np.unique(indexList)
    indexList = np.sort(indexList)    
    
    colList = np.append(colList, df.columns.values)
    colList = np.unique(colList)
    colList = np.sort(colList)
    

## Generate the DF for double index stacking
df = pd.DataFrame(index=indexList,columns=colList, data=0)
temp = df.stack()

## To drop the unwanted 0 column
temp.to_csv(Dir_Model + " _model.csv")
df = pd.read_csv(Dir_Model + "_model.csv")

T = df.drop("0", axis=1)

#FINAL MODEL
T.to_csv(Dir_Model + "model.csv", index=False)   
os.remove(Dir_Model + "_model.csv")
    

# Prepare csv of X-type variables (Multithread)

In [ ]:
import multiprocessing
import time
import glob
import numpy as np
import pandas as pd
import gc

n_cores = multiprocessing.cpu_count()
filesList = glob.glob(Dir_X + "*.csv")
data = filesList

def mp_worker(file_):
    process = multiprocessing.current_process()
    print "File: " + str(file_) 
    
    
    f_c = file_.replace('.csv','')
    f_c = f_c.replace(Dir_X,'')
    f_c_ = ''.join(e for e in f_c if e.isalnum())


    test = pd.read_csv(file_, index_col=0)
    
    #@TODO: Rustine to BE DROPPED once fixed in data source
    if 'CIQID' in test.columns:
        del test['CIQID']
    
    #test = test.fillna(-1)
    test = test.stack(dropna=False).to_frame()

    model = pd.read_csv(Dir_Model + "model.csv", index_col=[0,1])

    df = pd.DataFrame(index=model.index.difference(test.index),data=np.nan,columns=[0])
    
    frames = [test, df]
    bDF = pd.concat(frames)
    bDF.sort_index()
    bDF.columns = [f_c_]

    #bDF.to_csv("X_output/standalone/"+f_c_+".csv",index=True)
    bDF.to_csv(Output_Dir_X + f_c_ + ".csv",index=False) 
    
    del df
    del bDF
    del model
    del test
    gc.collect()
    
p = multiprocessing.Pool(Multithread_gran)
p.map(mp_worker, data)

In [ ]:
import multiprocessing
import time
import glob
import numpy as np
import pandas as pd
import gc
from functools import partial

def mp_worker(file_, modelDir, workingDir, outputDir):
    process = multiprocessing.current_process()
    print "File: " + str(file_) 
    
    
    f_c = file_.replace('.csv','')
    f_c = f_c.replace(workingDir,'')
    f_c_ = ''.join(e for e in f_c if e.isalnum())
    
    model = pd.read_csv(modelDir + "model.csv", index_col=[0,1])
    model[f_c_] = np.nan
    
    
    
    f = pd.read_csv(file_, index_col=0)
    if 'CIQID' in f.columns:
        del f['CIQID']
        
    f = f.stack().to_frame()
    f.columns = [f_c_]
    
    model.update(f)
    model.sort_index()

    model.to_csv(outputDir + f_c_ + ".csv",index=False) 
    
    del f
    del model
    gc.collect()


n_cores = multiprocessing.cpu_count()
filesList = glob.glob(Dir_X + "*.csv")
data = filesList
    
func_partX = partial(mp_worker, modelDir=Dir_Model, workingDir=Dir_X, outputDir=Output_Dir_X)
func_partY = partial(mp_worker, modelDir=Dir_Model, workingDir=Dir_Y, outputDir=Output_Dir_Y)

p = multiprocessing.Pool(Multithread_gran)
p.map(func_partX, data)


filesList = glob.glob(Dir_Y + "*.csv")
data = filesList
p.map(func_partY, data)

In [ ]:
filesList = glob.glob(Dir_Y + "*.csv")
data = filesList
p.map(func_partY, data)

# Prepare csv of Y-type variables (multithread)

In [ ]:
import multiprocessing
import time
import glob
import numpy as np
import pandas as pd
import gc

n_cores = multiprocessing.cpu_count()
filesList = glob.glob(Dir_Y + "*.csv")
data = filesList

def mp_worker(file_):
    process = multiprocessing.current_process()
    print "File: " + str(file_) 
    
    
    f_c = file_.replace('.csv','')
    f_c = f_c.replace(Dir_Y,'')
    f_c_ = ''.join(e for e in f_c if e.isalnum())


    test = pd.read_csv(file_, index_col=0)
    
    #@TODO: Rustine to BE DROPPED once fixed in data source
    if 'CIQID' in test.columns:
        del test['CIQID']
    
    #test = test.fillna(-1)
    test = test.stack(dropna=False).to_frame()

    model = pd.read_csv(Dir_Model + "model.csv", index_col=[0,1])

    df = pd.DataFrame(index=model.index.difference(test.index),data=np.nan,columns=[0])
   
    frames = [test, df]
    bDF = pd.concat(frames)
    bDF.sort_index()
    bDF.columns = [f_c_]

    #bDF.to_csv("X_output/standalone/"+f_c_+".csv",index=True)
    bDF.to_csv(Output_Dir_Y + f_c_ + ".csv",index=False) 
    
    del df
    del bDF
    del model
    del test
    gc.collect()
    
p = multiprocessing.Pool(Multithread_gran)
p.map(mp_worker, data)

 ## Append all CSV together - merged_X, header_X, merged_Y, header_Y (HDFS ready)

In [ ]:
from glob import glob
import numpy as np
import pandas as pd

ListF = glob(Output_Dir_X + '*.csv')
ListFiltered = []

i = 0
total = len(ListF)
print "X packages\n"

with open(Export_name + '_X.csv', 'w') as singleFile:
    for csv in ListF :
        i = i + 1 
        print "Work: #" + str(i) + " / " + str(total) 
        if csv == Output_Dir_X + Export_name + '_X.csv':
            pass
        else:
            ListFiltered.append(csv.replace(Output_Dir_X, '').replace('.csv', ''))
            for line in open(csv, 'r'):
                singleFile.write(line)
                
arr = np.array(ListFiltered)
df = pd.DataFrame(data=arr)
df.to_csv(Export_name + "_X_header.csv",index=False, header=False)

print "Y packages\n"
ListF = glob(Output_Dir_Y + '*.csv')
ListFiltered = []

i = 0
total = len(ListF)
with open(Export_name + '_Y.csv', 'w') as singleFile:
    for csv in ListF :
        i = i + 1 
        print "Work: #" + str(i) + " / " + str(total) 
        if csv == Output_Dir_Y + Export_name + '_Y.csv':
            pass
        else:
            ListFiltered.append(csv.replace(Output_Dir_Y, '').replace('.csv', ''))
            for line in open(csv, 'r'):
                singleFile.write(line)
                
arr = np.array(ListFiltered)
df = pd.DataFrame(data=arr)
df.to_csv(Export_name + "_Y_header.csv",index=False, header=False)

# GENERATE SHELL CODE TO EXPORT TO HDFS

In [ ]:
Source_Dir_X = Output_Dir_X
Source_Dir_Y = Output_Dir_Y


#import os
print "############################################################################"
print "##### Commandes à input dans un shell d'une machine connectée à Hadoop #####"
print "############################################################################"
print "" 

print 'if $(! hdfs dfs -test -d ' + HDFS_Dir + ') ; then hdfs dfs -mkdir -p ' + HDFS_Dir + '; fi'
print 'if $(! hdfs dfs -test -d ' + HDFS_Dir + ') ; then hdfs dfs -mkdir -p ' + HDFS_Dir + '; fi'

print ""
print "hdfs dfs -put -f " + Export_name + "_X.csv " + HDFS_Dir
print "hdfs dfs -put -f " + Export_name + "_X_header.csv " + HDFS_Dir
print "hdfs dfs -put -f " + Export_name + "_Y.csv " + HDFS_Dir
print "hdfs dfs -put -f " + Export_name + "_Y_header.csv " + HDFS_Dir
print "print 'done'"
#os.system("ls")

# GENERATE SHELL CODE TO EXPORT TO HDFS (Experimental only)

In [ ]:
Source_Dir_X = Output_Dir_X
Source_Dir_Y = Output_Dir_Y


#import os
print "############################################################################"
print "##### Commandes à input dans un shell d'une machine connectée à Hadoop #####"
print "############################################################################"
print "" 

print "#############################################################################################"
print "##### CES COMMANDES UPLOAD TOUS LES PETITS FICHIERS ET NON PAS LE GROS FICHIER FUSIONNE #####"
print "#############################################################################################"
print "" 

print 'if $(! hdfs dfs -test -d ' + HDFS_Dir_X + ') ; then hdfs dfs -mkdir ' + HDFS_Dir_X + '; fi'
print 'if $(! hdfs dfs -test -d ' + HDFS_Dir_Y + ') ; then hdfs dfs -mkdir ' + HDFS_Dir_Y + '; fi'

print ""
print "hdfs dfs -put -f " + Source_Dir_X +"*.csv " + HDFS_Dir_X 
print "hdfs dfs -put -f " + Source_Dir_Y +"*.csv " + HDFS_Dir_Y 

#os.system("ls")